**Merge separate datasets**

Jacqueline Simeone, Annie Bishai, Ankita Patil

In [1]:
import pandas as pd
import numpy as np

In [45]:
checkpoint = pd.read_csv('../clean_data/checkpoints_rev.csv')
svi = pd.read_csv('../clean_data/svi_clean.csv')
vaccine = pd.read_csv('../clean_data/vaccine_hesitancy.csv')
election = pd.read_csv('../clean_data/final_clean_vote_data.csv')
census = pd.read_csv('../clean_data/census_data.csv')

In [46]:
checkpoint.head(2)

,Recip_County,FIPS,Dose1_Recip_030121,Dose1_Pop_Pct_030121,Series_Complete_Pop_Pct_030121,Booster_Doses_Vax_Pct_030121,Dose1_Recip_090121,Dose1_Pop_Pct_090121,Series_Complete_Pop_Pct_090121,Booster_Doses_Vax_Pct_090121,Dose1_Recip_030122,Dose1_Pop_Pct_030122,Series_Complete_Pop_Pct_030122,Booster_Doses_Vax_Pct_030122
0,mason_mi,26105,5795.0,19.9,12.1,0,0.0,0.0,51.7,0,17777.0,61.0,60.1,58.2
1,colusa_ca,06011,2666.0,12.4,3.8,0,11806.0,54.8,45.8,0,14465.0,67.1,59.7,33.6


In [47]:
svi.head(2)

,index,socio_std,hh_std,minority_std,housing_std,overall_std,null_pct,uninsured_pct,socio_wgt,hh_wgt,minority_wgt,housing_wgt,overall_wgt,count
0,abbeville_sc,0.089387,0.195687,0.160383,0.134782,0.130340,0.0,0.115140,0.799646,0.793354,0.352214,0.674209,0.752302,6
1,acadia parish_la,0.159638,0.262590,0.232193,0.216910,0.223026,0.0,0.113333,0.784556,0.737875,0.323409,0.709167,0.731175,12


In [48]:
vaccine.head(2)

,county_state,n_zipcodes,pct_hesitant_02-24-22,pct_somewhat_hesitant_02-24-22,pct_highly_hesitant_02-24-22
0,abbeville_sc,13.0,0.293,0.108,0.185
1,acadia parish_la,15.0,0.290,0.083,0.207


In [49]:
election.head(2)

,county_state,state,candidate_won,party,winner_votes,total_votes,pct_trump,pct_biden
0,kent_de,DE,Joe Biden,DEM,44552,87025,0.488,0.512
1,new castle_de,DE,Joe Biden,DEM,195034,287633,0.322,0.678


In [50]:
census.head(2)

,County_State,County,State,SVI_CTGY,Metro_status,Census2019,Census2019_65PlusPop,Census2019_18to64Pop
0,mason_mi,Mason County,MI,2.0,0.0,29144.0,7069.0,16171.0
1,colusa_ca,Colusa County,CA,3.0,0.0,21547.0,3214.0,12492.0


In [51]:
#census.isnull().sum()
#went through columns with nans; the mismatched indices are not due to nans
#census[census['Census2019_18to64Pop'].isnull()]

In [52]:
# match format of "County_State" columns

vaccine.rename(columns={'county_state':'County_State'}, inplace=True)
election.rename(columns={'county_state':'County_State'}, inplace=True)
checkpoint.rename(columns={'Recip_County':'County_State'}, inplace=True)
svi.rename(columns={'index':'County_State'}, inplace=True)

In [53]:
#create list of each df's county_state's
checkpoint_county_list = list(checkpoint['County_State'])
svi_county_list = list(svi['County_State'])
vaccine_county_list = list(vaccine['County_State'])
election_county_list = list(election['County_State'])
census_county_list = list(census['County_State'])

lists = [checkpoint_county_list, svi_county_list, vaccine_county_list,
        election_county_list, census_county_list]

In [54]:
#lengths of lists
#
def length(lst, name):
    print(f'{name} length: {len(lst)}')

length(checkpoint_county_list, 'checkpoint')
length(svi_county_list, 'svi')
length(vaccine_county_list, 'vaccine')
length(election_county_list, 'election')
length(census_county_list, 'census')

checkpoint length: 3191
svi length: 3142
vaccine length: 3142
election length: 3145
census length: 3191


In [55]:
#print shapes
dfs = [checkpoint,svi,vaccine,election,census]

def shapes(df, dfname):
    print(f'{dfname} shape: {df.shape}')
    
def total_shape(df_list):
    count = 0
    for i in df_list:
        count += i.shape[1]
    print(f'total columns={count}')

In [56]:
shapes(checkpoint,'checkpoint')
shapes(svi, 'svi')
shapes(vaccine, 'vaccine')
shapes(election, 'election')
shapes(census, 'census')
total_shape(dfs)

checkpoint shape: (3191, 14)
svi shape: (3142, 14)
vaccine shape: (3142, 5)
election shape: (3145, 8)
census shape: (3191, 8)
total columns=49


In [57]:
#set indices to 'County_State'
#checked and they're all indexed correctly
for i in dfs:
    i.set_index('County_State', inplace=True)

In [58]:
# Check indices of checkpoint & census dfs are identical (3191 rows)

checkpt_has = [x for x in checkpoint.index if x not in census.index]
census_has = [x for x in census.index if x not in checkpoint.index]
print(checkpt_has)
print(census_has)

[]
[]


In [59]:
#merge: checkpoint, census (3191 rows, incl "unknown")

df_checkpt_census = pd.merge(checkpoint,census, left_index=True, right_index=True)
print(df_checkpt_census.shape)
# df_checkpt_census.head()

(3191, 20)


In [60]:
df_checkpt_census.loc[['dona ana_nm','la salle parish_la']]
df_checkpt_census.rename({'la salle parish_la': 'lasalle parish_la'}, inplace=True)

In [61]:
# Check indices of vaccine & svi dfs are identical (3142 rows)

svi_has = [x for x in svi.index if x not in vaccine.index]
vax_has = [x for x in vaccine.index if x not in svi.index]
print(svi_has)
print(vax_has)

['doña ana_nm', 'kalawao_hi', 'la salle parish_la', 'valdez-cordova_ak']
['chugach census area_ak', 'copper river census area_ak', 'do�a ana_nm', 'lasalle parish_la']


In [62]:
svi.rename({'doña ana_nm': 'dona ana_nm', 'la salle parish_la': 'lasalle parish_la'}, inplace=True)
vaccine.rename({'do�a ana_nm': 'dona ana_nm'}, inplace=True)

# Remaining mismatches: ['kalawao_hi', 'valdez-cordova_ak'] in svi
                    # ['chugach census area_ak', 'copper river census area_ak'] in vaccine
# df_checkpt_census.loc[['kalawao_hi', 'valdez-cordova_ak', 'chugach census area_ak', 'copper river census area_ak']]   
# These are not in election or df_checkpt_census

In [63]:
# Outer join to keep mismatched indices from both dataframes

df_vax_svi = pd.merge(left=vaccine, right=svi, how='outer', left_on='County_State', right_on='County_State')

df_vax_svi.shape

(3144, 17)

In [64]:
# Outer join of vax, svi (3142 rows) and checkpt, census (3191 rows)

df_vax_svi_checkpt_census = pd.merge(left=df_vax_svi, right=df_checkpt_census, how='outer', left_on='County_State', right_on='County_State')
df_vax_svi_checkpt_census.shape

(3195, 37)

In [65]:
# Check index mismatches btwn df_vax_svi_checkpt and election dfs

combined_has = [x for x in df_vax_svi_checkpt_census.index if x not in election.index and x[:7] != 'unknown']
election_has = [x for x in election.index if x not in df_vax_svi_checkpt_census.index]
print(f'combined_has: {combined_has}')
print(f'election_has: {election_has}')

combined_has: ['dona ana_nm', 'kalawao_hi']
election_has: ['do�a ana_nm']


In [66]:
# Correct dona ana_nm in election df
election.rename({'do�a ana_nm': 'dona ana_nm'}, inplace=True)

In [67]:
# Add kalawao_hi election info using wikipedia

hi = pd.DataFrame([{
     'state': 'HI',
     'candidate_won': 'Joe Biden',
     'party': 'DEM',
     'winner_votes': 23,
     'total_votes': 24,
     'pct_trump': .0417,
     'pct_biden': .8583, 
    'county_election': 'kalawao_hi'
}])

election_with_hi = pd.concat([election, hi])
election_with_hi.rename({0: 'kalawao_hi'}, inplace=True)
election_with_hi[election_with_hi['state'] == 'HI']


,state,candidate_won,party,winner_votes,total_votes,pct_trump,pct_biden,county_election
hawaii_hi,HI,Joe Biden,DEM,58731,87814,0.3310,0.6690,NaN
honolulu_hi,HI,Joe Biden,DEM,238869,382114,0.3750,0.6250,NaN
kauai_hi,HI,Joe Biden,DEM,21225,33497,0.3660,0.6340,NaN
maui_hi,HI,Joe Biden,DEM,47305,71044,0.3340,0.6660,NaN
kalawao_hi,HI,Joe Biden,DEM,23,24,0.0417,0.8583,kalawao_hi


In [68]:
election_with_hi.head()

,state,candidate_won,party,winner_votes,total_votes,pct_trump,pct_biden,county_election
kent_de,DE,Joe Biden,DEM,44552,87025,0.488,0.512,NaN
new castle_de,DE,Joe Biden,DEM,195034,287633,0.322,0.678,NaN
district of columbia_dc,DC,Joe Biden,DEM,39041,41681,0.063,0.937,NaN
alachua_fl,FL,Joe Biden,DEM,89704,142604,0.371,0.629,NaN
broward_fl,FL,Joe Biden,DEM,618752,957672,0.354,0.646,NaN


In [69]:
# Double-check indices before merging
combined_has = [x for x in df_vax_svi_checkpt_census.index if x not in election_with_hi.index]
election_has = [x for x in election_with_hi.index if x not in df_vax_svi_checkpt_census.index]
print(f'combined_has: {combined_has}')
print(f'election_with_hi_has: {election_has}')

combined_has: ['unknown_al', 'unknown_nc', 'unknown_wy', 'unknown_va', 'unknown_ia', 'unknown_nv', 'unknown_ms', 'unknown_az', 'unknown_nm', 'unknown_vt', 'unknown_mt', 'unknown_de', 'unknown_ut', 'unknown_ks', 'unknown_id', 'unknown_ak', 'unknown_or', 'unknown_mo', 'unknown_ok', 'unknown_ri', 'unknown_wi', 'unknown_sc', 'unknown_nd', 'unknown_ma', 'unknown_wv', 'unknown_pa', 'unknown_ct', 'unknown_dc', 'unknown_md', 'unknown_sd', 'unknown_tn', 'unknown_ar', 'unknown_ca', 'unknown_nh', 'unknown_ny', 'unknown_co', 'unknown_la', 'unknown_mi', 'unknown_fl', 'unknown_ne', 'unknown_ky', 'unknown_nj', 'unknown_wa', 'unknown_ga', 'unknown_in', 'unknown_oh', 'unknown_il', 'unknown_mn', 'unknown_me']
election_with_hi_has: []


In [70]:
election_with_hi.sort_index(inplace=True)
df_vax_svi_checkpt_census.sort_index(inplace=True)

election_with_hi['county_election'] = election_with_hi.index

In [71]:
# fix spelling of dona ana in order to use 'county_election' col as merge key
election_with_hi.loc['dona ana_nm', 'county_election'] = 'dona ana_nm'
election_with_hi.rename(columns={'county_election': 'County_State'}, inplace=True)

In [72]:
full_df = pd.merge(left=df_vax_svi_checkpt_census, right=election_with_hi, how='outer', left_on='County_State', right_on='County_State')
full_df.set_index('County_State', inplace=True)
print(full_df.shape)
full_df.head()

(3195, 44)


,n_zipcodes,pct_hesitant_02-24-22,pct_somewhat_hesitant_02-24-22,pct_highly_hesitant_02-24-22,socio_std,hh_std,minority_std,housing_std,overall_std,null_pct,...,Census2019,Census2019_65PlusPop,Census2019_18to64Pop,state,candidate_won,party,winner_votes,total_votes,pct_trump,pct_biden
County_State,,,,,,,,,,,,,,,,,,,,,
abbeville_sc,13.0,0.293,0.108,0.185,0.089387,0.195687,0.160383,0.134782,0.130340,0.000000,...,24527.0,5483.0,14184.0,SC,Donald Trump,REP,8215.0,12433.0,0.661,0.339
acadia parish_la,15.0,0.290,0.083,0.207,0.159638,0.262590,0.232193,0.216910,0.223026,0.000000,...,62045.0,9796.0,36244.0,LA,Donald Trump,REP,22596.0,28425.0,0.795,0.205
accomack_va,33.0,0.214,0.087,0.126,0.118796,0.108441,0.167966,0.172741,0.131746,0.272727,...,32316.0,7934.0,17692.0,VA,Donald Trump,REP,9172.0,16962.0,0.541,0.459
ada_id,17.0,0.165,0.037,0.128,0.201416,0.222791,0.164793,0.303244,0.227123,0.000000,...,481587.0,71958.0,297899.0,ID,Donald Trump,REP,130699.0,259389.0,0.504,0.496
adair_ia,13.0,0.290,0.059,0.231,0.032519,0.205443,0.055959,0.228846,0.139366,0.000000,...,7152.0,1648.0,3945.0,IA,Donald Trump,REP,2917.0,4177.0,0.698,0.302


In [73]:
# Create interaction term btwn hesitancy measures and # of zip codes averaged

full_df['hesitant_x_nzip'] = full_df['pct_hesitant_02-24-22'] * full_df['n_zipcodes']
full_df['somewhat_hesitant_x_nzip'] = full_df['pct_somewhat_hesitant_02-24-22'] * full_df['n_zipcodes']
full_df['highly_hesitant_x_nzip'] = full_df['pct_highly_hesitant_02-24-22'] * full_df['n_zipcodes']

In [74]:
# Make 'candidate_won' numeric
full_df['candidate_won'] = full_df['candidate_won'].map({'Donald Trump':1, 'Joe Biden':0})

In [75]:
# Add % over 65 columns
full_df['Census2019_Pct_65Plus'] = full_df['Census2019_65PlusPop'] / full_df['Census2019']

In [76]:
# Drop unwanted columns

full_df.drop(columns=['party', 'count', 'County', 'State', 'total_votes', 'winner_votes', 'socio_std', 'hh_std', 'housing_std', 'overall_std','minority_std', 
                      'Dose1_Recip_030121', 'Dose1_Recip_090121', 'Dose1_Recip_030122', 'n_zipcodes'], inplace=True)


In [77]:
full_df.to_csv('../clean_data/full_merged_data.csv')

In [78]:
full_df.isna().sum()

pct_hesitant_02-24-22             53
pct_somewhat_hesitant_02-24-22    53
pct_highly_hesitant_02-24-22      53
null_pct                          53
uninsured_pct                     53
socio_wgt                         53
hh_wgt                            53
minority_wgt                      53
housing_wgt                       53
overall_wgt                       53
FIPS                               4
Dose1_Pop_Pct_030121               4
Series_Complete_Pop_Pct_030121     4
Booster_Doses_Vax_Pct_030121       4
Dose1_Pop_Pct_090121               4
Series_Complete_Pop_Pct_090121     4
Booster_Doses_Vax_Pct_090121       4
Dose1_Pop_Pct_030122               4
Series_Complete_Pop_Pct_030122     4
Booster_Doses_Vax_Pct_030122       4
SVI_CTGY                          54
Metro_status                      54
Census2019                        53
Census2019_65PlusPop              53
Census2019_18to64Pop              53
state                             49
candidate_won                     49
p

In [79]:
full_df.shape

(3195, 33)